In [1]:
import glob
import pandas as pd
import numpy as np


In [3]:
glob.glob("*.csv")

['2016-08-18_IstDaten.csv',
 '2016-08-19_IstDaten.csv',
 '2016-08-20_IstDaten.csv',
 '2016-08-21_IstDaten.csv',
 '2016-08-22_IstDaten.csv',
 '2016-08-23_IstDaten.csv',
 '2016-08-25_IstDaten.csv',
 '2016-08-26_IstDaten.csv',
 '2016-08-27_IstDaten.csv',
 '2016-08-28_IstDaten.csv',
 '2016-08-29_IstDaten.csv',
 '2016-08-30_IstDaten.csv',
 '2016-08-31_IstDaten.csv',
 '2016-09-01_IstDaten.csv',
 '2016-09-02_IstDaten.csv',
 '2016-09-03_IstDaten.csv',
 '2016-09-04_IstDaten.csv',
 '2016-09-05_IstDaten.csv',
 '2016-09-07_IstDaten.csv',
 '2016-09-08_IstDaten.csv',
 '2016-09-09_IstDaten.csv',
 '2016-09-10_IstDaten.csv',
 '2016-09-11_IstDaten.csv',
 '2016-09-12_IstDaten.csv',
 '2016-09-13_IstDaten.csv',
 '2016-09-14_IstDaten.csv',
 '2016-09-15_IstDaten.csv',
 '2016-09-17_IstDaten.csv',
 '2016-09-18_IstDaten.csv',
 '2016-09-19_IstDaten.csv',
 '2016-09-20_IstDaten.csv',
 '2016-09-21_IstDaten.csv',
 '2016-09-22_IstDaten.csv',
 '2016-09-23_IstDaten.csv',
 '2016-09-24_IstDaten.csv',
 '2016-09-25_IstDate